In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.applications.resnet50 import ResNet50
from keras.models import load_model
import tensorflow as tf
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from keras.utils import plot_model

Using TensorFlow backend.


In [2]:
train_path = 'X:/Flipkart GRID - Object Localization (Medium Complexity Round)/augument_data/training_images'

img_width, img_height = 640., 480.

In [3]:
##### img_width, img_height = 640, 480
base_dir = 'X:/Flipkart GRID - Object Localization (Medium Complexity Round)/'
# train_data_dir = "X:/train-scene classification/data/train/"
# validation_data_dir = "X:/train-scene classification/data/valid/"
# test_data_dir = "X:/train-scene classification/data/test/"
epochs = 1000
# Save the model according to the conditions  
chkpt_dir = "X:/Flipkart GRID - Object Localization (Medium Complexity Round)/chkpts/resnet50model-2-norm_io.h5" # Change for every new training session 

In [4]:
# base_model = ResNet50(weights = "imagenet", include_top=False, input_shape = (img_height, img_width, 3), classes = 4)

In [5]:
# base_model.save('resnet50.h5')
base_model = load_model(os.path.join(base_dir,'base_models', 'resnet50.h5'))

C:\Users\praty\Anaconda3\envs\dlenv\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [6]:
for layer in base_model.layers[:-5]:
    layer.trainable = False
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 480, 640, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 486, 646, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 240, 320, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 240, 320, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [18]:
# for layer in base_model.layers:
#     layer.trainable = False
x = base_model.output
x = Flatten()(x)
# x = Dense(2048, activation="relu")(x)
# x = Dropout(0.7)(x)
x = Dense(32, activation="relu")(x)
x = Dropout(0.2)(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.1)(x)
# x = Dense(64, activation="relu")(x)
# x = Dropout(0.0)(x)
# x = Dense(1024, activation="relu")(x)
# x = Dropout(0.5)(x)
# x = Dense(512, activation="relu")(x)
# x = Dropout(0.3)(x)
# predictions = Dense(6, activation="softmax")(x)
predictions = Dense(4, activation="linear")(x)

In [19]:
model = Model(input = base_model.input, output = predictions)
# for layer in model.layers[:-3]:
#     layer.trainable = False
model.summary()

C:\Users\praty\Anaconda3\envs\dlenv\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """Entry point for launching an IPython kernel.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 480, 640, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 486, 646, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 240, 320, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 240, 320, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

bn5a_branch2b (BatchNormalizati (None, 15, 20, 512)  2048        res5a_branch2b[0][0]             
__________________________________________________________________________________________________
activation_42 (Activation)      (None, 15, 20, 512)  0           bn5a_branch2b[0][0]              
__________________________________________________________________________________________________
res5a_branch2c (Conv2D)         (None, 15, 20, 2048) 1050624     activation_42[0][0]              
__________________________________________________________________________________________________
res5a_branch1 (Conv2D)          (None, 15, 20, 2048) 2099200     activation_40[0][0]              
__________________________________________________________________________________________________
bn5a_branch2c (BatchNormalizati (None, 15, 20, 2048) 8192        res5a_branch2c[0][0]             
__________________________________________________________________________________________________
bn5a_branc

In [20]:
def mIoU(labels,predictions):
#     print(labels.shape)
#     print(predictions.shape)
#     print(labels[0])
#     print(predictions[0])
    pred_x1=predictions[:,0]#*img_width
    pred_x2=predictions[:,1]#*img_width
    pred_y1=predictions[:,2]#*img_height
    pred_y2=predictions[:,3]#*img_height
    label_x1=labels[:,0]#*img_width
    label_x2=labels[:,1]#*img_width
    label_y1=labels[:,2]#*img_height
    label_y2=labels[:,3]#*img_height
    int_x1 = tf.maximum(pred_x1, label_x1)
    int_x2 = tf.minimum(pred_x2, label_x2)
    int_y1 = tf.maximum(pred_y1, label_y1)
    int_y2 = tf.minimum(pred_y2, label_y2)
    tensor_type = pred_x1.dtype
    pred_area = tf.multiply(tf.maximum(tf.cast(0.0, tensor_type), pred_x2 - pred_x1), tf.maximum(tf.cast(0.0, tensor_type), pred_y2 - pred_y1))
    label_area = tf.multiply(tf.maximum(tf.cast(0.0, tensor_type), label_x2 - label_x1), tf.maximum(tf.cast(0.0, tensor_type), label_y2 - label_y1))
    int_area = tf.multiply(tf.maximum(tf.cast(0.0, tensor_type), int_x2 - int_x1), tf.maximum(tf.cast(0.0, tensor_type), int_y2 - int_y1))
    union_area = pred_area + label_area - int_area
    IoU = int_area / union_area
    return tf.reduce_mean(IoU)

In [21]:
# IoU Metric Test
LABEL = [[0.0, 1.0, 0.0, 1.0], 
        [2.0, 3.0, 2.0, 3.0]]
PRED = [[0.5, 1.5, 0.5, 1.5],
        [2.0, 3.0, 2.0, 3.0]]
LABEL = tf.constant(np.array(LABEL))
PRED = tf.constant(np.array(PRED))
print(mIoU(LABEL, PRED))
with tf.Session() as sess:
    val = sess.run(mIoU(LABEL, PRED))
    print(val)

Tensor("Mean_2:0", shape=(), dtype=float64)
0.5714285714285714


#### Load best model weights from here, if required.

In [22]:
# model.load_weights(chkpt_dir)

In [23]:
model.compile(loss = "mse", optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8), metrics=['mse', mIoU])

In [24]:
checkpoint = ModelCheckpoint(chkpt_dir, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=40, verbose=1, mode='auto')

In [25]:
train_df = pd.read_csv(os.path.join(base_dir, 'augument_data', 'augmented_train_data.csv'))
val_df = pd.read_csv(os.path.join(os.path.join(base_dir, 'augument_data', 'augmented_val_data.csv')))

In [15]:
l = []
for im in val_df.image_name:
    img = (cv2.imread(os.path.join(train_path, im)) / 255.).astype(np.float32) ### INTRIDUCING NORMALIZATION.
    l.append(np.expand_dims(img, axis = 0))
x_val = np.vstack(l)
print("x_val done.")
y_val = val_df.iloc[:,1:].values.astype(np.float32)

y_val[:, 0] = y_val[:, 0] # / img_width
y_val[:, 1] = y_val[:, 1] # / img_width
y_val[:, 2] = y_val[:, 2] # / img_height
y_val[:, 3] = y_val[:, 3] # / img_height
print(y_val[0])
print(y_val.dtype)
print(x_val.shape)

x_val done.
[ 78. 578.  78. 317.]
float32
(2000, 480, 640, 3)


In [26]:
model.load_weights(chkpt_dir)

In [27]:
ram_batch_size = 500
total_number_images = 68000
for epoch in range(epochs):
    print(epoch)
    print("--------------------------------------------------------------")
    for i in range(total_number_images // ram_batch_size):
        print('batch {}/135 (epoch {}):'.format(i, epoch))
        l = []
        for im in train_df.image_name[i * ram_batch_size: (i + 1) * ram_batch_size]:
            img = (cv2.imread(os.path.join(train_path, im)) / 255.).astype(np.float32) ### INTRIDUCING NORMALIZATION.
            l.append(np.expand_dims(img, axis = 0))
        x_train = np.vstack(l)
        y_train = train_df[i * ram_batch_size: (i + 1) * ram_batch_size].iloc[:,1:].values
        y_train[:, 0] = y_train[:, 0] # / img_width
        y_train[:, 1] = y_train[:, 1] # / img_width
        y_train[:, 2] = y_train[:, 2] # / img_height
        y_train[:, 3] = y_train[:, 3] # / img_height
        
        model.fit(x=x_train, y=y_train, batch_size=32, epochs=1, verbose=1, callbacks=[checkpoint], validation_split=0.0, validation_data=(x_val, y_val), shuffle=True)

0
--------------------------------------------------------------
batch 0/135 (epoch 0):
Train on 500 samples, validate on 2000 samples
Epoch 1/1
500/500 [==============================] - 52s 104ms/step - loss: 21791.0332 - mean_squared_error: 21791.0332 - mIoU: 0.2511 - val_loss: 13754.1935 - val_mean_squared_error: 13754.1935 - val_mIoU: 0.2916

Epoch 00001: val_loss improved from inf to 13754.19352, saving model to X:/Flipkart GRID - Object Localization (Medium Complexity Round)/chkpts/resnet50model-2-norm_io.h5
batch 1/135 (epoch 0):
Train on 500 samples, validate on 2000 samples
Epoch 1/1
500/500 [==============================] - 50s 100ms/step - loss: 18333.7898 - mean_squared_error: 18333.7898 - mIoU: 0.3137 - val_loss: 17692.1062 - val_mean_squared_error: 17692.1062 - val_mIoU: 0.2060

Epoch 00001: val_loss did not improve from 13754.19352
batch 2/135 (epoch 0):
Train on 500 samples, validate on 2000 samples
Epoch 1/1
500/500 [==============================] - 49s 99ms/step - 

Train on 500 samples, validate on 2000 samples
Epoch 1/1
500/500 [==============================] - 49s 98ms/step - loss: 7209.5524 - mean_squared_error: 7209.5524 - mIoU: 0.5003 - val_loss: 4461.3940 - val_mean_squared_error: 4461.3940 - val_mIoU: 0.5535

Epoch 00001: val_loss did not improve from 4363.18727
batch 23/135 (epoch 0):
Train on 500 samples, validate on 2000 samples
Epoch 1/1
500/500 [==============================] - 49s 99ms/step - loss: 8326.0189 - mean_squared_error: 8326.0189 - mIoU: 0.4794 - val_loss: 4435.8772 - val_mean_squared_error: 4435.8772 - val_mIoU: 0.5569

Epoch 00001: val_loss did not improve from 4363.18727
batch 24/135 (epoch 0):
Train on 500 samples, validate on 2000 samples
Epoch 1/1
500/500 [==============================] - 49s 99ms/step - loss: 7256.1559 - mean_squared_error: 7256.1559 - mIoU: 0.5006 - val_loss: 5948.2222 - val_mean_squared_error: 5948.2222 - val_mIoU: 0.4891

Epoch 00001: val_loss did not improve from 4363.18727
batch 25/135 (epoch

500/500 [==============================] - 49s 99ms/step - loss: 5980.3301 - mean_squared_error: 5980.3301 - mIoU: 0.5372 - val_loss: 14528.6969 - val_mean_squared_error: 14528.6969 - val_mIoU: 0.2703

Epoch 00001: val_loss did not improve from 4363.18727
batch 47/135 (epoch 0):
Train on 500 samples, validate on 2000 samples
Epoch 1/1
500/500 [==============================] - 49s 99ms/step - loss: 5931.6362 - mean_squared_error: 5931.6362 - mIoU: 0.5300 - val_loss: 18331.3870 - val_mean_squared_error: 18331.3870 - val_mIoU: 0.2079

Epoch 00001: val_loss did not improve from 4363.18727
batch 48/135 (epoch 0):
Train on 500 samples, validate on 2000 samples
Epoch 1/1
500/500 [==============================] - 49s 99ms/step - loss: 5313.2507 - mean_squared_error: 5313.2507 - mIoU: 0.5518 - val_loss: 13505.3321 - val_mean_squared_error: 13505.3321 - val_mIoU: 0.2904

Epoch 00001: val_loss did not improve from 4363.18727
batch 49/135 (epoch 0):
Train on 500 samples, validate on 2000 samples

500/500 [==============================] - 49s 99ms/step - loss: 5555.8370 - mean_squared_error: 5555.8370 - mIoU: 0.5524 - val_loss: 8091.0056 - val_mean_squared_error: 8091.0056 - val_mIoU: 0.4316

Epoch 00001: val_loss did not improve from 4363.18727
batch 71/135 (epoch 0):
Train on 500 samples, validate on 2000 samples
Epoch 1/1
500/500 [==============================] - 49s 98ms/step - loss: 5721.8609 - mean_squared_error: 5721.8609 - mIoU: 0.5342 - val_loss: 10211.6486 - val_mean_squared_error: 10211.6486 - val_mIoU: 0.3671

Epoch 00001: val_loss did not improve from 4363.18727
batch 72/135 (epoch 0):
Train on 500 samples, validate on 2000 samples
Epoch 1/1
500/500 [==============================] - 49s 99ms/step - loss: 5538.9150 - mean_squared_error: 5538.9150 - mIoU: 0.5602 - val_loss: 9302.2699 - val_mean_squared_error: 9302.2699 - val_mIoU: 0.3997

Epoch 00001: val_loss did not improve from 4363.18727
batch 73/135 (epoch 0):
Train on 500 samples, validate on 2000 samples
Epo


Epoch 00001: val_loss did not improve from 4363.18727
batch 95/135 (epoch 0):
Train on 500 samples, validate on 2000 samples
Epoch 1/1
500/500 [==============================] - 49s 98ms/step - loss: 4661.6556 - mean_squared_error: 4661.6556 - mIoU: 0.5771 - val_loss: 9341.4897 - val_mean_squared_error: 9341.4897 - val_mIoU: 0.3847

Epoch 00001: val_loss did not improve from 4363.18727
batch 96/135 (epoch 0):
Train on 500 samples, validate on 2000 samples
Epoch 1/1
500/500 [==============================] - 49s 99ms/step - loss: 5288.4257 - mean_squared_error: 5288.4257 - mIoU: 0.5636 - val_loss: 10228.2692 - val_mean_squared_error: 10228.2692 - val_mIoU: 0.3507

Epoch 00001: val_loss did not improve from 4363.18727
batch 97/135 (epoch 0):
Train on 500 samples, validate on 2000 samples
Epoch 1/1
500/500 [==============================] - 49s 99ms/step - loss: 5332.5921 - mean_squared_error: 5332.5921 - mIoU: 0.5547 - val_loss: 9997.1754 - val_mean_squared_error: 9997.1754 - val_mIoU: 

...

500/500 [==============================] - 49s 99ms/step - loss: 3759.3695 - mean_squared_error: 3759.3695 - mIoU: 0.6060 - val_loss: 11574.9593 - val_mean_squared_error: 11574.9593 - val_mIoU: 0.3363

Epoch 00001: val_loss did not improve from 4363.18727
batch 108/135 (epoch 4):
Train on 500 samples, validate on 2000 samples
Epoch 1/1
500/500 [==============================] - 49s 99ms/step - loss: 3443.1691 - mean_squared_error: 3443.1691 - mIoU: 0.6133 - val_loss: 11080.3304 - val_mean_squared_error: 11080.3304 - val_mIoU: 0.3538

Epoch 00001: val_loss did not improve from 4363.18727
batch 109/135 (epoch 4):
Train on 500 samples, validate on 2000 samples
Epoch 1/1
500/500 [==============================] - 49s 98ms/step - loss: 3507.9506 - mean_squared_error: 3507.9506 - mIoU: 0.6081 - val_loss: 9974.0449 - val_mean_squared_error: 9974.0449 - val_mIoU: 0.3772

Epoch 00001: val_loss did not improve from 4363.18727
batch 110/135 (epoch 4):
Train on 500 samples, validate on 2000 sample

500/500 [==============================] - 49s 99ms/step - loss: 3497.8965 - mean_squared_error: 3497.8965 - mIoU: 0.6162 - val_loss: 11306.4616 - val_mean_squared_error: 11306.4616 - val_mIoU: 0.3546

Epoch 00001: val_loss did not improve from 4363.18727
batch 132/135 (epoch 4):
Train on 500 samples, validate on 2000 samples
Epoch 1/1
500/500 [==============================] - 49s 99ms/step - loss: 3523.2028 - mean_squared_error: 3523.2028 - mIoU: 0.6212 - val_loss: 9855.6268 - val_mean_squared_error: 9855.6268 - val_mIoU: 0.3879

Epoch 00001: val_loss did not improve from 4363.18727
batch 133/135 (epoch 4):
Train on 500 samples, validate on 2000 samples
Epoch 1/1
500/500 [==============================] - 49s 99ms/step - loss: 3523.4452 - mean_squared_error: 3523.4452 - mIoU: 0.6103 - val_loss: 9622.1412 - val_mean_squared_error: 9622.1412 - val_mIoU: 0.3935

Epoch 00001: val_loss did not improve from 4363.18727
batch 134/135 (epoch 4):
Train on 500 samples, validate on 2000 samples


500/500 [==============================] - 49s 99ms/step - loss: 3109.0356 - mean_squared_error: 3109.0356 - mIoU: 0.6254 - val_loss: 7856.4371 - val_mean_squared_error: 7856.4371 - val_mIoU: 0.4552

Epoch 00001: val_loss did not improve from 4363.18727
batch 20/135 (epoch 5):
Train on 500 samples, validate on 2000 samples
Epoch 1/1
500/500 [==============================] - 49s 99ms/step - loss: 3455.4892 - mean_squared_error: 3455.4892 - mIoU: 0.6220 - val_loss: 9129.8765 - val_mean_squared_error: 9129.8765 - val_mIoU: 0.4190

Epoch 00001: val_loss did not improve from 4363.18727
batch 21/135 (epoch 5):
Train on 500 samples, validate on 2000 samples
Epoch 1/1
500/500 [==============================] - 49s 99ms/step - loss: 3557.6936 - mean_squared_error: 3557.6936 - mIoU: 0.6021 - val_loss: 9767.3648 - val_mean_squared_error: 9767.3648 - val_mIoU: 0.4023

Epoch 00001: val_loss did not improve from 4363.18727
batch 22/135 (epoch 5):
Train on 500 samples, validate on 2000 samples
Epoch


Epoch 00001: val_loss did not improve from 4363.18727
batch 44/135 (epoch 5):
Train on 500 samples, validate on 2000 samples
Epoch 1/1
500/500 [==============================] - 50s 99ms/step - loss: 3701.4650 - mean_squared_error: 3701.4650 - mIoU: 0.6144 - val_loss: 10399.0611 - val_mean_squared_error: 10399.0611 - val_mIoU: 0.3737

Epoch 00001: val_loss did not improve from 4363.18727
batch 45/135 (epoch 5):
Train on 500 samples, validate on 2000 samples
Epoch 1/1
500/500 [==============================] - 50s 99ms/step - loss: 3340.9371 - mean_squared_error: 3340.9371 - mIoU: 0.6120 - val_loss: 12056.6804 - val_mean_squared_error: 12056.6804 - val_mIoU: 0.3392

Epoch 00001: val_loss did not improve from 4363.18727
batch 46/135 (epoch 5):
Train on 500 samples, validate on 2000 samples
Epoch 1/1
500/500 [==============================] - 50s 99ms/step - loss: 3387.9335 - mean_squared_error: 3387.9335 - mIoU: 0.6291 - val_loss: 10302.7789 - val_mean_squared_error: 10302.7789 - val_mI

KeyboardInterrupt: 

In [ ]:
model.save('transfer_resnet50-2.h5')

In [ ]:
model.load_weights(chkpt_dir)
LABEL = y_val
PRED = model.predict(x_val)
with tf.Session() as sess:
    val = sess.run(mIoU(LABEL, PRED))
    print(val)